In [108]:
# import modules
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import scipy.signal
from scipy import stats
from scipy.io import wavfile
import numpy as np
import math
from sklearn.metrics import pairwise_distances

bandLimits = [0, 200, 400, 800, 1600, 3200, 6400]    
combFilterPulses = 8
minBpm = 60
maxBpm = 240

In [109]:
y, sr = librosa.load('Data/genres_original/pop/pop.00001.wav', offset=10)
y_comp, sr = librosa.load('Data/genres_original/pop/pop.00001.wav', offset=20) 

# y, sr = librosa.load('RileyKaraoke.wav', offset=20, duration=20)
# y_comp, sr = librosa.load('RileyKaraoke.wav', offset=10, duration=20) 

In [110]:
tempo, beat_times = librosa.beat.beat_track(y, sr=sr, units='time')
tempo, comp_beat_times = librosa.beat.beat_track(y_comp, sr=sr, units='time')

hop_length = 512
frame_length = 1024
 
chroma_ref = librosa.feature.chroma_cqt(y=y, sr=sr, hop_length=hop_length)
chroma_comp = librosa.feature.chroma_cqt(y=y_comp, sr=sr, hop_length=hop_length)

mfcc_ref = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length)
mfcc_comp = librosa.feature.mfcc(y=y_comp, sr=sr, hop_length=hop_length)

# Use time-delay embedding to get a cleaner recurrence matrix
x_ref = librosa.feature.stack_memory(chroma_ref, n_steps=10, delay=3)
x_comp = librosa.feature.stack_memory(chroma_comp, n_steps=10, delay=3)

/Users/pianistprogrammer/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/pianistprogrammer/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):


In [114]:
#  corr = scipy.correlate(beat_times, comp_beat_times,  mode='full')
# corr = librosa.segment.cross_similarity(chroma_ref, chroma_comp, metric='cosine', mode='distance')
# corr
corr =  1-pairwise_distances(mfcc_ref, metric="cosine")
 

In [105]:
def detect_tempo(signal, accuracy: int, minBpm: int, maxBpm: int, bandsLimits, sr,
                     combFilterPulses):
        n = len(signal[0])
        bands_amount = len(bandsLimits)
        dft = np.zeros([bands_amount, n], dtype=complex)

        if minBpm < 60:
            minBpm = 60

        if maxBpm > 240:
            maxBpm = 240

        for band in range(0, bands_amount):
            dft[band] = np.fft.fft(signal[band])

        maxEnergy = 0
        for bpm in range(minBpm, maxBpm, accuracy):
            this_bpm_energy = 0
            fil = np.zeros(n)

            filter_step = np.floor(60 / bpm * sr)
            percent_done = 100 * (bpm - minBpm) / (maxBpm - minBpm)
            print("%.2f" % percent_done, "%")

            for a in range(0, combFilterPulses):
                fil[a * int(filter_step) + 1] = 1

            dftfil = np.fft.fft(fil)

            for band in range(0, bands_amount):
                x = (abs(dftfil * dft[band])) ** 2
                this_bpm_energy = this_bpm_energy + sum(x)

            if this_bpm_energy > maxEnergy:
                songBpm = bpm
                maxEnergy = this_bpm_energy

        return songBpm

In [111]:
__methods = []
    
def detect_metre(signal, tempo: int, bandlimits, sr):
        nbands = len(bandlimits)

        __methods.append(__five_forth)
        __methods.append(__four_forth)
        __methods.append(__six_eigth)
        __methods.append(__three_forth)

        metres = {}
        for method in __methods:
            metre, metre_dft = method(tempo, sr)
            metres[metre] = metre_dft

        maxe = 0
        done = 0
        todo = len(metres.keys())
        for metrum in metres:
            done += 1
            percent_done = 100 * done / todo
            print("%.2f" % percent_done, "%")

            e = 0
            for band in range(0, nbands):
                filt = scipy.correlate(signal[band], metres[metrum])
                f_filt = abs(np.fft.fft(filt))
                x = abs(f_filt) ** 2
                e = e + sum(x)

            if e > maxe:
                song_metre = metrum
                maxe = e

        return song_metre
    
def __four_forth(song_tempo: int, sampling_frequency: int):
        fil = np.zeros(int(4 * sampling_frequency * (60 / song_tempo)))
        nstep = np.floor(60 / song_tempo * sampling_frequency)

        value = 1 / 2
        fil[int(1 * nstep)] = 1 * value
        fil[int(3 * nstep)] = 1 * value

        return "4\\4", fil

def __three_forth(song_tempo: int, sampling_frequency: int):
        fil = np.zeros(int(6 * sampling_frequency * (60 / song_tempo)))
        nstep = np.floor(60 / song_tempo * sampling_frequency)

        value = 1 / 2
        fil[int(2 * nstep)] = 1 * value
        fil[int(5 * nstep)] = 1 * value

        return "3\\4", fil

def __five_forth(song_tempo: int, sampling_frequency: int):
        fil = np.zeros(int(5 * sampling_frequency * (60 / song_tempo)))
        nstep = np.floor(60 / song_tempo * sampling_frequency)

        value = 1 / 3
        fil[int(1 * nstep)] = 1 * value
        fil[int(3 * nstep)] = 1 * value
        fil[int(4 * nstep)] = 1 * value

        return "5\\4", fil

def __six_eigth(song_tempo: int, sampling_frequency: int):
        fil = np.zeros(int(3 * sampling_frequency * (60 / song_tempo)))
        nstep = np.floor((60 / song_tempo * sampling_frequency) / 2)

        value = 1 / 2
        fil[int(0 * nstep)] = 1 * value
        fil[int(3 * nstep)] = 1 * value

        return "6\\8", fil

In [115]:
metre = detect_metre(corr, tempo, bandLimits, sr)
print ('meter is', metre)

25.00 %
50.00 %
75.00 %
100.00 %
meter is 3\4
